In [ ]:
def xmlCleaner(line):
    """
    return plain text from a line of XML
    """
    while '<' in line:
        line = line.strip()
        #print("loop 1",end=' ')
        s = line.find('<')
        e = line.find('>') + 1
        try:
            while '<' == line[e]:
                #print("loop 2",end=' ')
                e = line[e:len(line)].find('>') + e +1
                #print(f'{e}')
        except IndexError:
            pass
        to_rep = line[s:e]
        #print(len(line),len(to_rep), sep='\n')
        if len(line)==len(to_rep):
            return ''
        #print(line,to_rep, sep='\n')
        line = line.replace(to_rep, ' ').strip()
    while '  ' in line:
        line = line.replace('  ',' ')
    while '- ' in line:
        line = line.replace('- ','\n').strip()
    return line

def helper():
    s = ''
    diz = json.load(open(py_wd + os.sep + 'shortcut.json', 'r', encoding="utf-8"))
    for k in diz.keys():
        if k.lower() in diz[k].lower():
            i = diz[k].lower().find(k.lower())
            s += diz[k][:i]+'('+diz[k][i].upper()+')'+diz[k][i+1:]+'\t'
        else:
            s += '('+k+')'+diz[k]+'\t'
        if len(diz[k]) < 6:
            s += '\t'
        #if ' ' in diz[k]:
        #    s += k+':'+diz[k][diz[k].find(' ')+1:len(diz[k])]+'\t'
        #else:
        #    s += k+':'+diz[k]+'\t'
    s += '\n\t:'
    print(s)
        
helper()

In [ ]:
def listBattute(file):
    """
    Return a list with all Battute in a episode
    """
    episode = list()
    for line in file:
        n_line = xmlCleaner(line)
        if len(n_line)>0:
            if '\n' in n_line:
                for row in n_line.split('\n'):
                    episode.append(row.strip())
            else:
                episode.append(n_line)
    return episode

In [ ]:
import os
import pandas as pd
################## GET full path for raw and py dir
py_wd = os.getcwd()
os.chdir('..')
os.chdir('./RAW')
raw_path = os.getcwd()
###################### CREATE df
df = pd.DataFrame(columns=['personaggio','battuta','stagione','episodio','index'])
######################

files = os.listdir()
for file in files:
    season = int(file[file.rfind('S')+1:file.rfind('S')+3])
    episode = int(file[file.rfind('E')+1:file.rfind('E')+3])
    if season < 2:
        print(file)
        fin = open(raw_path+os.sep+file, encoding='UTF-8')
        list_battute = listBattute(fin)
        for i in range(0,len(list_battute)):
            df = df.append({
                'personaggio': str(),
                'battuta': list_battute[i],
                'stagione': season,
                'episodio': episode,
                'index': i+1
            },ignore_index=True)
os.chdir(py_wd)
df = df.sort_values(by=['episodio','index'],kind='heapsort',ignore_index=True)
df.to_csv(py_wd+os.sep+'Suburra_data.csv', index=False, encoding='UTF-8')

In [ ]:

#fin = open("/home/smoxy/Scrivania/Naldi/AIMLEA/AIMLEA4Suburra/RAW/SuburraS01E07.xml", encoding='UTF-8')
#c = 0
#for line in fin:
#    n_line = xmlCleaner(line)
#    if len(n_line)>0:
#        #print(line, end='')
#        print(n_line)
#    c += 1
#    if c==20:
#        print("finish")
#        break

In [110]:
df = pd.read_csv(py_wd+os.sep+'Suburra_data.csv', encoding='UTF-8')
df.head(16)


,personaggio,battuta,stagione,episodio,index
0,radio,"[radio] Oggi, dopo sette anni di mandato,",1,1,1
1,radio,il sindaco di Roma ha presentato le proprie di...,1,1,2
2,radio,che saranno effettive entro 20 giorni.,1,1,3
3,radio,[radio] Nel suo discorso di addio all'ultima r...,1,1,4
4,radio,ha ribadito come la sua scelta sia stata molto...,1,1,5
5,radio,[radio] Il sindaco uscente ha ricordato,1,1,6
6,radio,che il rispetto per gli elettori e per i citta...,1,1,7
7,radio,è sempre stato messo al primo posto.,1,1,8
8,radio,[radio] Adesso si prospetta uno scenario di fo...,1,1,9
9,radio,"per tutti gli schieramenti politici,",1,1,10


In [104]:
################ LABELLING
def labelling(df, start=0):
    for i in range(start,len(df)):
        print('\n',i,'\t',df.loc[i,'battuta'], sep='')
        label = input(helper()).lower()
        if len(label)==0:
            label = df.loc[i-1,'personaggio']
            df.loc[i,'personaggio'] = label
        elif label[0].upper() in diz.keys() and len(label)<3:
            label = diz[label[0].upper()]
        elif label == '0':
            df.to_csv(py_wd+os.sep+'Suburra_data.csv', index=False, encoding='UTF-8')
            break
        else:
            df.loc[i,'personaggio'] = label
        os.system('clear')
        print('\t',label)
start = int(input("da quale battuta vuoi ripartire? "))
labelling(df)

ValueError: invalid literal for int() with base 10: ''